# Geospatial Analysis

In [ ]:
# Run this cell to load the tdquiz magic command, that runs queries and return the results.
%load_ext tdquiz

# Test tdquiz magic command
%tdquiz SELECT USER, SESSION, CURRENT_TIMESTAMP

<h3>Distance between two Offices</h3><span class="tag" style="font-size: smaller; background-color: #dddddd; color: #222222">geospatial</span> <span class="tag" style="font-size: smaller; background-color: #dddddd; color: #222222">distance</span>
<br>
<em>facilities.teradata_offices</em> table stores the locations of Teradata offices.
Two of them are in Japan.
Calculate the distance between the two Japan offices in kilometers.
Note that we use the <a href="https://en.wikipedia.org/wiki/Great-circle_distance" rel="noopener noreferrer" target="_blank">Great-cirle distance formula</a> for this calculation.
<details><summary>Hint</summary><ul>
<li><a href="https://docs.teradata.com/r/Teradata-VantageTM-Geospatial-Data-Types-17.20/ST_Point-Methods/ST_SphericalDistance" rel="noopener noreferrer" target="_blank">Geospatial Data Types (manual)</a>
</ul></details>
<details><summary>Answer</summary><pre style="margin: 1em 0; padding: 1em; border-radius: 5px; background-color: #25292f; white-space: pre-wrap;"><code style="background-color: #25292f; color: #ffffff">SELECT
  (NEW ST_Geometry('ST_Point', a.longitude, a.latitude)).ST_SphericalDistance(
    NEW ST_Geometry('ST_Point', b.longitude, b.latitude)) / 1000 AS distance_km
FROM
  facilities.teradata_offices AS a
  INNER JOIN facilities.teradata_offices AS b
    ON a.office_name < b.office_name
       AND a.country = 'jpn'
       AND b.country = 'jpn'</code></pre></details>

In [ ]:
%%tdquiz
/* Write your query blow */






<h3>Shinkansen station in a village</h3><span class="tag" style="font-size: smaller; background-color: #dddddd; color: #222222">geospatial</span> <span class="tag" style="font-size: smaller; background-color: #dddddd; color: #222222">join</span>
<br>
Find all shinkansen stations located in a village, and display the line name, the prefecture name, the village name and the station name.
<details><summary>Hint</summary><ul>
<li><em>japan.cities</em>, <em>facilities.stations</em>, <em>facilities.shinkansen</em>
<li>What method would be relevant for geometries being contained?
<li><a href="https://docs.teradata.com/r/Teradata-VantageTM-Geospatial-Data-Types/July-2021/ST_Geometry-Constructors-and-Methods" rel="noopener noreferrer" target="_blank">Geometry methods (manual)</a>
</ul></details>
<details><summary>Answer</summary><pre style="margin: 1em 0; padding: 1em; border-radius: 5px; background-color: #25292f; white-space: pre-wrap;"><code style="background-color: #25292f; color: #ffffff">SELECT
  s.line_name,
  c.pref_name,
  c.city_name,
  s.station_name
FROM
  facilities.stations AS s
  INNER JOIN japan.cities AS c
    ON c.geometry.ST_INTERSECTS(s.geometry) = 1
WHERE 
  s.line_name IN ( SELECT DISTINCT line_name FROM facilities.shinkansen )
  AND RIGHT(c.city_name, 1) = '村'</code></pre></details>

In [ ]:
%%tdquiz
/* Write your query blow */






<h3>Stations at ends</h3><span class="tag" style="font-size: smaller; background-color: #dddddd; color: #222222">geospatial</span>
<br>
What are the northmost, southmost, westmost, and eastmost stations in Japan?
<details><summary>Hint</summary><ul>
<li>Check out <em>ST_X, ST_Y</em>.
<li><a href="https://docs.teradata.com/r/Teradata-VantageTM-Geospatial-Data-Types/July-2021/ST_Geometry-Constructors-and-Methods" rel="noopener noreferrer" target="_blank">Geometry methods (manual)</a>
<li><a href="https://tabi-mag.jp/station-news/" rel="noopener noreferrer" target="_blank">Maybe after you solve this!</a>
</ul></details>
<details><summary>Answer</summary><pre style="margin: 1em 0; padding: 1em; border-radius: 5px; background-color: #25292f; white-space: pre-wrap;"><code style="background-color: #25292f; color: #ffffff">with ends AS (
  SELECT
    MIN(geometry.ST_X()) AS east_end,
    MAX(geometry.ST_X()) AS west_end,
    MIN(geometry.ST_Y()) AS south_end,
    MAX(geometry.ST_Y()) AS north_end
  FROM
    facilities.stations
)
SELECT
  north.station_name AS northmost,
  south.station_name AS southmost,
  west.station_name AS westmost,
  east.station_name AS eastmost
FROM
  ends
  INNER JOIN facilities.stations AS east ON east.geometry.ST_X() = ends.east_end
  INNER JOIN facilities.stations AS west ON west.geometry.ST_X() = ends.west_end
  INNER JOIN facilities.stations AS north ON north.geometry.ST_Y() = ends.north_end
  INNER JOIN facilities.stations AS south ON south.geometry.ST_Y() = ends.south_end</code></pre></details>

In [ ]:
%%tdquiz
/* Write your query blow */






<h3>Prefectures where shinkansen passes through</h3><span class="tag" style="font-size: smaller; background-color: #dddddd; color: #222222">geospatial</span> <span class="tag" style="font-size: smaller; background-color: #dddddd; color: #222222">join</span>
<br>
Extract prefectures where shinkansen visits, and calculate the number of shinkansen station in them.
Are there any prefectures where shinkansen just passes through with stopping on no station?
Display the prefecture name and the number of shinkansen stations, sorted by the number of the stations and the prefecture name.
<details><summary>Hint</summary><ul>
<li><em>japan.cities</em>, <em>facilities.stations</em>, <em>facilities.shinkansen</em>
<li>What method would be relevant for geometries being contained?
<li><a href="https://docs.teradata.com/r/Teradata-VantageTM-Geospatial-Data-Types/July-2021/ST_Geometry-Constructors-and-Methods" rel="noopener noreferrer" target="_blank">Geometry methods (manual)</a>
<li><a href="https://news.mynavi.jp/article/trivia-435/" rel="noopener noreferrer" target="_blank">Maybe after you solve the question!</a>
</ul></details>
<details><summary>Answer</summary><pre style="margin: 1em 0; padding: 1em; border-radius: 5px; background-color: #25292f; white-space: pre-wrap;"><code style="background-color: #25292f; color: #ffffff">WITH tmp AS (
  SELECT DISTINCT
    p.pref_name
  FROM
    japan.prefectures AS p
    INNER JOIN facilities.shinkansen AS s
      ON p.geometry.ST_INTERSECTS(s.geometry) = 1
),
tmp2 AS (
  SELECT
    p.pref_name, count(*) AS num_stations
  FROM
    facilities.stations AS s
    INNER JOIN japan.prefectures AS p
      ON p.geometry.ST_INTERSECTS(s.geometry) = 1
  WHERE
    s.line_name IN ( SELECT DISTINCT line_name FROM facilities.shinkansen )
  GROUP BY 1
)
SELECT
  a.pref_name,
  COALESCE(b.num_stations, 0) AS num_stations
FROM
  tmp AS a LEFT JOIN tmp2 AS b ON a.pref_name = b.pref_name
ORDER BY 2,1</code></pre></details>

In [ ]:
%%tdquiz
/* Write your query blow */






<h3>Disjoint cities</h3><span class="tag" style="font-size: smaller; background-color: #dddddd; color: #222222">geospatial</span>
<br>
Some municipalities consists of multiple disjoint parts.
Show the names of all municipalites in Gumma (群馬) prefecture.
<details><summary>Hint</summary><ul>
<li>Use <em>japan.cities</em>
<li><a href="https://docs.teradata.com/r/Teradata-VantageTM-Geospatial-Data-Types/July-2021/ST_Geometry-Constructors-and-Methods/ST_GeometryType-Method" rel="noopener noreferrer" target="_blank">ST_GeometryType (マニュアル)</a>
<li>What geometry type represents disjoint parts with areas?
<li><a href="https://ja.wikipedia.org/wiki/%E9%A3%9B%E5%9C%B0" rel="noopener noreferrer" target="_blank">飛び地 (Wikipedia)</a>
</ul></details>
<details><summary>Answer</summary><pre style="margin: 1em 0; padding: 1em; border-radius: 5px; background-color: #25292f; white-space: pre-wrap;"><code style="background-color: #25292f; color: #ffffff">SELECT
  pref_name, city_name
FROM
  japan.cities
WHERE
  pref_name = '群馬県'
  AND geometry.ST_GeometryType() = 'ST_MultiPolygon'</code></pre></details>

In [ ]:
%%tdquiz
/* Write your query blow */






<h3>All geometry columns
</h3><span class="tag" style="font-size: smaller; background-color: #dddddd; color: #222222">teradata-specific</span> <span class="tag" style="font-size: smaller; background-color: #dddddd; color: #222222">ddl</span>
<br>
Show all columns of geometry type (ST_Geometry) in the tables in the database <em>japan</em>, <em>facilities</em>, and <em>TeraShirt</em>.
Display the database name, the table name, and the geometry column name.
<details><summary>Hint</summary>Check out <em>dbc.ColumnsV</em>.</details>
<details><summary>Answer</summary><pre style="margin: 1em 0; padding: 1em; border-radius: 5px; background-color: #25292f; white-space: pre-wrap;"><code style="background-color: #25292f; color: #ffffff">SELECT
  databaseName, tableName, columnName
FROM
  dbc.ColumnsV
WHERE
  columnUDTName = 'ST_GEOMETRY'
  AND databaseName IN ('japan', 'facilities', 'TeraShirt')</code></pre></details>

In [ ]:
%%tdquiz
/* Write your query blow */




